In [4]:
import duckdb

con = duckdb.connect('tpch/dev.duckdb')
con.sql("install tpch; load tpch")
con.sql('CALL dbgen(sf=1);')
con.close()


In [3]:
con.close()

In [16]:
con = duckdb.connect('tpch/dev.duckdb')
con.sql("describe stg_orders")

┌─────────────────────┬───────────────┬─────────┬─────────┬─────────┬───────┐
│     column_name     │  column_type  │  null   │   key   │ default │ extra │
│       varchar       │    varchar    │ varchar │ varchar │ varchar │ int32 │
├─────────────────────┼───────────────┼─────────┼─────────┼─────────┼───────┤
│ order_key           │ INTEGER       │ YES     │ NULL    │ NULL    │  NULL │
│ order_cust_key      │ INTEGER       │ YES     │ NULL    │ NULL    │  NULL │
│ order_status        │ VARCHAR       │ YES     │ NULL    │ NULL    │  NULL │
│ order_total_price   │ DECIMAL(15,2) │ YES     │ NULL    │ NULL    │  NULL │
│ order_date          │ DATE          │ YES     │ NULL    │ NULL    │  NULL │
│ order_priority      │ VARCHAR       │ YES     │ NULL    │ NULL    │  NULL │
│ order_clerk         │ VARCHAR       │ YES     │ NULL    │ NULL    │  NULL │
│ order_ship_priority │ INTEGER       │ YES     │ NULL    │ NULL    │  NULL │
│ order_comment       │ VARCHAR       │ YES     │ NULL    │ NULL

In [18]:
con.sql("describe stg_customer")

┌──────────────────────┬───────────────┬─────────┬─────────┬─────────┬───────┐
│     column_name      │  column_type  │  null   │   key   │ default │ extra │
│       varchar        │    varchar    │ varchar │ varchar │ varchar │ int32 │
├──────────────────────┼───────────────┼─────────┼─────────┼─────────┼───────┤
│ cust_key             │ INTEGER       │ YES     │ NULL    │ NULL    │  NULL │
│ cust_name            │ VARCHAR       │ YES     │ NULL    │ NULL    │  NULL │
│ cust_address         │ VARCHAR       │ YES     │ NULL    │ NULL    │  NULL │
│ cust_nation_key      │ INTEGER       │ YES     │ NULL    │ NULL    │  NULL │
│ cust_phone           │ VARCHAR       │ YES     │ NULL    │ NULL    │  NULL │
│ cust_account_balance │ DECIMAL(15,2) │ YES     │ NULL    │ NULL    │  NULL │
│ cust_market_segment  │ VARCHAR       │ YES     │ NULL    │ NULL    │  NULL │
│ cust_comment         │ VARCHAR       │ YES     │ NULL    │ NULL    │  NULL │
└──────────────────────┴───────────────┴─────────┴──

In [7]:
con = duckdb.connect('tpch/dev.duckdb', read_only= True)
con.sql('SELECT * FROM stg_lineitem')

┌──────────────────────┬────────────────────┬───┬──────────────────────┬────────────────────┬──────────────────────┐
│     lineitem_key     │ lineitem_order_key │ … │ lineitem_ship_inst…  │ lineitem_ship_mode │   lineitem_comment   │
│       varchar        │       int32        │   │       varchar        │      varchar       │       varchar        │
├──────────────────────┼────────────────────┼───┼──────────────────────┼────────────────────┼──────────────────────┤
│ ea66c06c1e1c05fa9f…  │                  1 │ … │ DELIVER IN PERSON    │ TRUCK              │ to beans x-ray car…  │
│ 98c6f2c2287f4c73ce…  │                  1 │ … │ TAKE BACK RETURN     │ MAIL               │  according to the …  │
│ 13cee27a2bd9391547…  │                  1 │ … │ TAKE BACK RETURN     │ REG AIR            │ ourts cajole above…  │
│ 3a170a9fe4f47efa37…  │                  1 │ … │ NONE                 │ AIR                │ s cajole busily ab…  │
│ 12426c956d1bc50170…  │                  1 │ … │ NONE          

In [70]:
query = '''
SELECT 
l_orderkey as lineitem_order_key, 
l_partkey as lineitem_part_key, 
l_suppkey as lineitem_supplied_key,
l_linenumber as lineitem_linenumber,
---- I'm gonna assume that any value that isn't 'N' is a return 
case when l_returnflag = 'N' then l_quantity else 0 - l_quantity end as lineitem_quantity,
l_extendedprice as lineitem_extended_price,
l_discount as lineitem_discount, 
l_tax as lineitem_tax, 
(l_extendedprice - l_discount) * (case when l_returnflag = 'N' then l_quantity else 0 - l_quantity end) as lineitem_line_total,
((l_extendedprice - l_discount) * (1 + l_tax)) * (case when l_returnflag = 'N' then l_quantity else 0 - l_quantity end) as lineitem_line_total_tax,
l_returnflag as lineitem_return_flag,
l_linestatus as lineitem_line_status,
l_shipdate as lineitem_ship_date, 
l_commitdate as lineitem_commit_date, 
l_receiptdate as lineitem_receipt_date,
l_shipinstruct as lineitem_ship_instruct,
l_shipmode as lineitem_ship_mode,
l_comment as lineitem_comment
from lineitem
'''

con.sql(query)

┌────────────────────┬───────────────────┬──────────────────────┬───┬────────────────────┬──────────────────────┐
│ lineitem_order_key │ lineitem_part_key │ lineitem_supplied_…  │ … │ lineitem_ship_mode │   lineitem_comment   │
│       int32        │       int32       │        int32         │   │      varchar       │       varchar        │
├────────────────────┼───────────────────┼──────────────────────┼───┼────────────────────┼──────────────────────┤
│                  1 │            155190 │                 7706 │ … │ TRUCK              │ to beans x-ray car…  │
│                  1 │             67310 │                 7311 │ … │ MAIL               │  according to the …  │
│                  1 │             63700 │                 3701 │ … │ REG AIR            │ ourts cajole above…  │
│                  1 │              2132 │                 4633 │ … │ AIR                │ s cajole busily ab…  │
│                  1 │             24027 │                 1534 │ … │ FOB               

In [71]:
con.close()